In [1]:
from os import environ
from dotenv import load_dotenv
import numpy as np
import pandas as pd
import time
from riotwatcher import LolWatcher, ApiError
from sklearn.datasets import load_iris
import plotly.express as px

load_dotenv("../")

RIOT_API = environ["RIOT_API"]
lol_watcher = LolWatcher(RIOT_API)
my_region = "na1"
NA = "na1"

In [2]:
yogurtsauce = pd.DataFrame.from_dict(
    lol_watcher.summoner.by_name(NA, "yogurtsauce"), orient="index"
).T

In [3]:
ranked = lol_watcher.league.by_summoner(NA, yogurtsauce.id[0])
versions = lol_watcher.data_dragon.versions_for_region(NA)
champions_version = versions["n"]["champion"]
summoner_spells_version = versions["n"]["summoner"]
items_version = versions["n"]["item"]
current_champ_list = lol_watcher.data_dragon.champions(champions_version)
my_matches = lol_watcher.match.matchlist_by_puuid(NA, yogurtsauce.puuid[0])
current_items = lol_watcher.data_dragon.items(items_version)
mastery_version = versions["n"]["mastery"]

In [222]:
champions = (
    pd.DataFrame.from_dict(current_champ_list)
    .reset_index()
    .rename(columns={"index": "champion"})
)

champions = pd.concat([champions, pd.json_normalize(champions.data)], axis=1).drop(
    columns=["type", "format", "version", "data", "name", "id"]
)

champions.insert(0, "key", champions.pop("key"))
champions[["primary_role", "secondary_role"]] = pd.DataFrame(
    champions.tags.tolist(), index=champions.index
)
champions = champions.drop(columns="tags")
champions.primary_role = champions.primary_role.astype("category")
champions.secondary_role = champions.secondary_role.astype("category")
champions.partype = champions.partype.astype("category")
champions.head()

,key,champion,title,blurb,partype,info.attack,info.defense,info.magic,info.difficulty,image.full,...,stats.mpregen,stats.mpregenperlevel,stats.crit,stats.critperlevel,stats.attackdamage,stats.attackdamageperlevel,stats.attackspeedperlevel,stats.attackspeed,primary_role,secondary_role
0,266,Aatrox,the Darkin Blade,Once honored defenders of Shurima against the ...,Blood Well,8,4,3,4,Aatrox.png,...,0.0,0.0,0,0,60,5.00,2.500,0.651,Fighter,Tank
1,103,Ahri,the Nine-Tailed Fox,Innately connected to the magic of the spirit ...,Mana,3,4,8,5,Ahri.png,...,8.0,0.8,0,0,53,3.00,2.000,0.668,Mage,Assassin
2,84,Akali,the Rogue Assassin,Abandoning the Kinkou Order and her title of t...,Energy,5,3,8,7,Akali.png,...,50.0,0.0,0,0,62,3.30,3.200,0.625,Assassin,NaN
3,166,Akshan,the Rogue Sentinel,"Raising an eyebrow in the face of danger, Aksh...",Mana,0,0,0,0,Akshan.png,...,8.2,0.7,0,0,52,3.50,4.000,0.638,Marksman,Assassin
4,12,Alistar,the Minotaur,Always a mighty warrior with a fearsome reputa...,Mana,6,9,5,7,Alistar.png,...,8.5,0.8,0,0,62,3.75,2.125,0.625,Tank,Support


In [5]:
class Summoner:
    def __init__(self, region, summoner_name):
        self.region = region
        self.name = summoner_name
        self.data = pd.DataFrame.from_dict(
            lol_watcher.summoner.by_name(self.region, self.name), orient="index"
        ).T
        self.id = self.data.id[0]
        self.account_id = self.data.accountId[0]
        self.puuid = self.data.puuid[0]
        self.profile_icon_id = self.data.profileIconId[0]
        self.revision_date = self.data.revisionDate[0]
        self.level = self.data.summonerLevel[0]
        self.matches_ = lol_watcher.match.matchlist_by_puuid(self.region, self.puuid)


yogurtsauce = Summoner("NA1", "yogurtsauce")

In [302]:
def match_history(username, region, n_games=5):
    i = 0
    j = 0
    user = Summoner(region, username)
    matches = pd.DataFrame()

    while j < n_games:
        try:
            last_match = user.matches_[j]
            match_details = lol_watcher.match.by_id(region, last_match)
            match_players = match_details["info"]["participants"]
            details = []

            for player in match_players:
                dict = {}
                dict["ign"] = player["summonerName"]
                dict["level"] = player["summonerLevel"]
                dict["champion_id"] = player["championId"]
                dict["champion"] = player["championName"]
                dict[
                    "mastery_points"
                ] = lol_watcher.champion_mastery.by_summoner_by_champion(
                    region, player["summonerId"], player["championId"]
                )[
                    "championPoints"
                ]
                dict["team"] = player["teamId"]
                dict["win"] = player["win"]

                details.append(dict)

            details = pd.DataFrame.from_dict(details)
            details = details.merge(
                champions[
                    [
                        "champion",
                        "primary_role",
                        "secondary_role",
                        "info.attack",
                        "info.defense",
                        "info.magic",
                        "info.difficulty",
                        "partype"
                    ]
                ]
            )
            
            details.team = details.team.astype("category")

            matches = pd.concat([matches, details], ignore_index=True)

            i += 1
            j += 1
        except:
            j += 1

    return matches


test = match_history("yogurtsauce", "na1", 1)

In [303]:
test

,ign,level,champion_id,champion,mastery_points,team,win,primary_role,secondary_role,info.attack,info.defense,info.magic,info.difficulty,partype
0,Chikoriiita,385,24,Jax,1820,100,True,Fighter,Assassin,7,5,7,5,Mana
1,I Counter Faker,216,236,Lucian,45732,100,True,Marksman,NaN,8,5,3,6,Mana
2,0ushi,538,37,Sona,28056,100,True,Support,Mage,5,2,8,4,Mana
3,Reasonably Salty,166,161,Velkoz,31787,100,True,Mage,NaN,2,2,10,8,Mana
4,yogurtsauce,514,147,Seraphine,10551,100,True,Mage,Support,0,0,0,0,Mana
5,Ttugizzing,670,163,Taliyah,1775,200,False,Mage,Support,1,7,8,5,Mana
6,Alseus,401,67,Vayne,21680,200,False,Marksman,Assassin,10,1,1,8,Mana
7,TOMBSTONE TIM,376,19,Warwick,21131,200,False,Fighter,Tank,9,5,3,3,Mana
8,Hunter301202,153,166,Akshan,23968,200,False,Marksman,Assassin,0,0,0,0,Mana
9,Lee Send,110,64,LeeSin,145228,200,False,Fighter,Assassin,8,5,3,6,Energy


In [283]:
wins = test.where(test.win == True)
wins = wins[~wins["ign"].isnull()]

losses = test.where(test.win == False)
losses = losses[~losses["ign"].isnull()]

In [304]:
wins

,ign,level,champion_id,champion,mastery_points,team,win,primary_role,secondary_role,info.attack,info.defense,info.magic,info.difficulty,partype
0,Chikoriiita,385.0,24.0,Jax,1820.0,100,True,Fighter,Assassin,7.0,5.0,7.0,5.0,Mana
1,I Counter Faker,216.0,236.0,Lucian,45732.0,100,True,Marksman,NaN,8.0,5.0,3.0,6.0,Mana
2,0ushi,538.0,37.0,Sona,28056.0,100,True,Support,Mage,5.0,2.0,8.0,4.0,Mana
3,Reasonably Salty,166.0,161.0,Velkoz,31787.0,100,True,Mage,NaN,2.0,2.0,10.0,8.0,Mana
4,yogurtsauce,514.0,147.0,Seraphine,10551.0,100,True,Mage,Support,0.0,0.0,0.0,0.0,Mana


In [290]:
losses.pivot(index = "win", columns="ign")

level                                                champion_id   
ign   Alseus Hunter301202 Lee Send TOMBSTONE TIM Ttugizzing      Alseus   
win                                                                       
False  401.0        153.0    110.0         376.0      670.0        67.0  \

                                                      ... info.difficulty   
ign   Hunter301202 Lee Send TOMBSTONE TIM Ttugizzing  ...          Alseus   
win                                                   ...                   
False        166.0     64.0          19.0      163.0  ...             8.0  \

                                                     partype                
ign   Hunter301202 Lee Send TOMBSTONE TIM Ttugizzing  Alseus Hunter301202   
win                                                                         
False          0.0      6.0           3.0        5.0    Mana         Mana  \

                                         
ign   Lee Send TOMBSTONE TIM Ttugizzing  
win                                      
False   Energy          Mana       Mana  

[1 rows x 60 columns]

In [301]:
wins

,ign,level,champion_id,champion,mastery_points,team,win,primary_role,secondary_role,info.attack,info.defense,info.magic,info.difficulty,partype
0,Chikoriiita,385.0,24.0,Jax,1820.0,100,True,Fighter,Assassin,7.0,5.0,7.0,5.0,Mana
1,I Counter Faker,216.0,236.0,Lucian,45732.0,100,True,Marksman,NaN,8.0,5.0,3.0,6.0,Mana
2,0ushi,538.0,37.0,Sona,28056.0,100,True,Support,Mage,5.0,2.0,8.0,4.0,Mana
3,Reasonably Salty,166.0,161.0,Velkoz,31787.0,100,True,Mage,NaN,2.0,2.0,10.0,8.0,Mana
4,yogurtsauce,514.0,147.0,Seraphine,10551.0,100,True,Mage,Support,0.0,0.0,0.0,0.0,Mana


In [243]:
test.pivot(index="win", columns=["ign"], values=["champion"])

champion                                                             
ign      0ushi Alseus Chikoriiita Hunter301202 I Counter Faker  Lee Send   
win                                                                        
False      NaN  Vayne         NaN       Akshan              NaN   LeeSin  \
True      Sona    NaN         Jax          NaN           Lucian      NaN   

                                                             
ign   Reasonably Salty TOMBSTONE TIM Ttugizzing yogurtsauce  
win                                                          
False              NaN       Warwick    Taliyah         NaN  
True            Velkoz           NaN        NaN   Seraphine

In [226]:
test.dtypes

ign                  object
level                 int64
champion_id           int64
champion             object
mastery_points        int64
team               category
win                    bool
primary_role       category
secondary_role     category
info.attack           int64
info.defense          int64
info.magic            int64
info.difficulty       int64
partype            category
dtype: object

In [211]:
champions

,key,champion,title,blurb,partype,info.attack,info.defense,info.magic,info.difficulty,image.full,...,stats.mpregen,stats.mpregenperlevel,stats.crit,stats.critperlevel,stats.attackdamage,stats.attackdamageperlevel,stats.attackspeedperlevel,stats.attackspeed,primary_role,secondary_role
0,266,Aatrox,the Darkin Blade,Once honored defenders of Shurima against the ...,Blood Well,8,4,3,4,Aatrox.png,...,0.00,0.00,0,0,60,5.00,2.500,0.651,Fighter,Tank
1,103,Ahri,the Nine-Tailed Fox,Innately connected to the magic of the spirit ...,Mana,3,4,8,5,Ahri.png,...,8.00,0.80,0,0,53,3.00,2.000,0.668,Mage,Assassin
2,84,Akali,the Rogue Assassin,Abandoning the Kinkou Order and her title of t...,Energy,5,3,8,7,Akali.png,...,50.00,0.00,0,0,62,3.30,3.200,0.625,Assassin,NaN
3,166,Akshan,the Rogue Sentinel,"Raising an eyebrow in the face of danger, Aksh...",Mana,0,0,0,0,Akshan.png,...,8.20,0.70,0,0,52,3.50,4.000,0.638,Marksman,Assassin
4,12,Alistar,the Minotaur,Always a mighty warrior with a fearsome reputa...,Mana,6,9,5,7,Alistar.png,...,8.50,0.80,0,0,62,3.75,2.125,0.625,Tank,Support
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
158,221,Zeri,The Spark of Zaun,"A headstrong, spirited young woman from Zaun's...",Mana,8,5,3,6,Zeri.png,...,6.00,0.80,0,0,53,1.30,2.000,0.658,Marksman,NaN
159,115,Ziggs,the Hexplosives Expert,"With a love of big bombs and short fuses, the ...",Mana,2,4,9,4,Ziggs.png,...,8.00,0.80,0,0,54,3.10,2.000,0.656,Mage,NaN
160,26,Zilean,the Chronokeeper,"Once a powerful Icathian mage, Zilean became o...",Mana,2,5,8,6,Zilean.png,...,11.35,0.80,0,0,52,3.00,2.130,0.625,Support,Mage
161,142,Zoe,the Aspect of Twilight,"As the embodiment of mischief, imagination, an...",Mana,1,7,8,5,Zoe.png,...,8.00,0.65,0,0,58,3.30,2.500,0.625,Mage,Support
